In [8]:
import pandas as pd
import numpy as np
import pygeohash as gh

In [9]:
file = pd.read_csv("../raw/mpv-data.csv", sep=",")
df =pd.DataFrame(file)
df = df.loc[:,['date', 'latitude', 'longitude']]
# print(df.shape)
# print(df.isna().sum())
df=df.dropna()
df['geohash']=df.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=5), axis=1)

print(df.head())
print(df.shape)

unique_geo=df.drop_duplicates('geohash')
print(unique_geo.shape)
unique_geo=unique_geo['geohash']

bins=range(len(df))
geo_list = unique_geo.tolist()

geo_dict= { geo_list[i] : i+1 for i in range(0, len(geo_list) ) }

df['bins']=df['geohash'].map(geo_dict)
df['rownum']=bins
df['LAG']=0

print(type(df['date']))
df['date']=pd.to_datetime(df['date'])
df['date']=df['date'].dt.strftime('%m/%d/%y')

geo_final = df.loc[:,['bins', 'date', 'LAG']]
geo_final = geo_final.rename({'bins': 'bin', 'date':'OCURRED'}, axis=1)
geo_final.to_csv('../final/mpv-geohashed.csv')
print(geo_final.shape)
print(geo_final.head())

        date   latitude   longitude geohash
0  6/23/2022  41.000000  -75.360000   dr63j
1  6/22/2022  42.250000  -84.760000   dpe8r
2  6/22/2022  37.412169 -121.960331   9q9kb
3  6/21/2022  36.891971  -76.302463   dq99z
4  6/21/2022  40.720000  -84.090000   dphpw
(10453, 4)
(6980, 4)
<class 'pandas.core.series.Series'>
(10453, 3)
   bin   OCURRED  LAG
0    1  06/23/22    0
1    2  06/22/22    0
2    3  06/22/22    0
3    4  06/21/22    0
4    5  06/21/22    0


In [5]:
def is_geohash_in_bounding_box(current_geohash, bbox_coords):
    
    coordinates=gh.decode(current_geohash)
    geohash_in_bounding_box=(bbox_coords[0]<coordinates[0]<bbox_coords[2]) and (bbox_coords[1]<coordinates[1] <bbox_coords[3])
    return geohash_in_bounding_box

def build_geohash_box(current_geohash):
    b=gh.bbox(current_geohash)
    polygon=[(b['w'], b['s']), (b['w'], b['n']), (b['e'], b['n']), (b['e'], b['s'],), (b['w'], b['s'])]
    return polygon

def compute_geohash_tiles(bbox_coordinates):
    """Computes all geohash tile in the given bounding box

    :param bbox_coordinates: the bounding box coordinates of the geohashes
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding box, assuming the earth is flat ;)
    center_latitude = (bbox_coordinates[0] + bbox_coordinates[2]) / 2
    center_longitude = (bbox_coordinates[1] + bbox_coordinates[3]) / 2

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            if neighbor not in checked_geohashes and is_geohash_in_bounding_box(neighbor, bbox_coordinates):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes

In [7]:
import geojson
from geojson import MultiLineString

def write_geohash_layer(geohashes):
    """Writes a grid layer based on the geohashes

    :param geohashes: a list of geohashes
    """

    layer = MultiLineString([build_geohash_box(gh) for gh in geohashes])
    with open('ghash_berlin_bbox.json', 'wb') as f:
        f.write(geojson.dumps(ghash_polygon, sort_keys=True).encode('utf-8'))
        
write_geohash_layer(geo_list)

AttributeError: module 'pygeohash' has no attribute 'bbox'

In [9]:
from shapely import geometry

def compute_geohash_tiles_from_polygon(polygon):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes